# Parameter sweeps

The `DifferentialEquations.jl` and `JuliaDynamics` ecosystems are useful
for making highly optimized parameter sweeps of dynamical systems.

In particular, we'll look at:
- `OrdinaryDiffEq` (DifferentialEquations)
- `ChaosTools` (JuliaDynamics)

The tools from JuliaDynamics' `DynamicalSystems.jl` (of which `ChaosTools.jl` is
a part) are documented in tutorial format very well in the book by Datseris &
Parlitz.

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

# Lyapunov spectrum analysis

We'll compute two sweeps:
- *Leading Lyapunov exponent*, which shows parameter regions corresponding to
  chaotic dynamics.
- *Lyapunov dimension*, which shows the fractal dimension of the attractor in the
  state space.

Let's compute these for a single point in parameter space to see how they work.

In [ ]:
using OrdinaryDiffEq

In [ ]:
function lorenz!(du, u, p, t)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u
  
  # Define the Lorenz system equations.
  du[1] = σ * (y - x)
  du[2] = x * (ρ - z) - y
  du[3] = x * y - β * z

  # Return the derivatives; we will need this for BifurcationKit.jl.
  du
end

In [ ]:
p = [
  28.0,   # ρ
  10.0,   # σ
  8.0/3.0 # β
]

u0 = [
  10.0, # x
  10.0, # y
  10.0  # z
]

tspan = (0.0, 1e5)

In [ ]:
# Integrate.
prob = ODEProblem(lorenz!, u0, tspan, p)
solution = solve(prob, Tsit5(), abstol=1e-6, reltol=1e-6, maxiters=1e8)

In [ ]:
# Plot solution.

using GLMakie

# Extract the solution components.
xs = [u[1] for u in solution.u]
ys = [u[2] for u in solution.u]
zs = [u[3] for u in solution.u]

# Create the 3D plot.
fig = Figure()
ax = Axis3(
  fig[1, 1], 
  xlabel = "x", 
  ylabel = "y", 
  zlabel = "z",
  title = "Lorenz system trajectory"
)

# Plot the solution.
lines!(ax, xs, ys, zs, linewidth = 1)

# Display the figure.
fig

ChaosTools.jl wants the system specified not as an `ODEProblem`, but instead as
a `DynamicalSystem`, which is a type defined by ChaosTools.jl.
The appropriate subtype of `DynamicalSystem` for the Lorenz system is
`CoupledODEs`.

In [ ]:
using ChaosTools

# Set up the system.
lorenz_system = CoupledODEs(
  lorenz!,
  u0,
  p
)

In [ ]:
# Compute the Lyapunov spectrum.
λ = lyapunovspectrum(
  lorenz_system, # The DynamicalSystem.
  Int(3e2);      # Integration steps.
  Ttr = 1e2,     # Transient time.
  Δt = 1e-1      # Time between orthonormalization steps.
)

In [ ]:
# Calculate the Lyapunov dimension (Kaplan-Yorke dimension).
# This is defined as k + sum(λs[1:k])/|λs[k+1]|, where k is the largest
# integer such that the sum of the first k Lyapunov exponents is non-negative.
function lyapunov_dimension(λ)
    k = 0
    sum_λ = 0.0
    
    # Find the largest k where the sum is still non-negative.
    for (i, λ) in enumerate(λ)
        sum_λ += λ
        if sum_λ < 0
            break
        end
        k = i
    end
    
    # If all exponents are positive or k is the last index, return k.
    if k == length(λ) || k == 0
        return k
    end
    
    # Calculate the Kaplan-Yorke dimension.
    return k + sum(λ[1:k])/abs(λ[k+1])
end

In [ ]:
lyap_dim = lyapunov_dimension(λ)
println("Estimated Lyapunov dimension: $lyap_dim")

We should expect a Lyapunov dimension of $2.06 \pm 0.01$.

# Sweeps over $\rho, \sigma$

In [ ]:
using StaticArrays

In [ ]:
# Define parameter range.
# ρs = range(10.0, 150.0, length=30);
ρs = range(0.0, 110.0, length=64);
# σs = range(0.0, 80.0, length=30);
σs = range(20.0, 55.0, length=64);

In [ ]:
# Parameter values from the Cartesian product ρs × σs × {β}.
params = [[ρ, σ, p[3]] for ρ in ρs for σ in σs]

In [ ]:
function lorenz_static(u, p, _)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u
  
  # Define the Lorenz system equations.
  du = SVector{3}(
    σ * (y - x),
    x * (ρ - z) - y,
    x * y - β * z
  )

  # Return the derivatives.
  du
end

In [ ]:
function lorenz_static_jac(u, p, _)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u

  # Define the Lorenz system Jacobian.
  jac = SMatrix{3, 3}(
     -σ,  σ,  0,
    ρ-z, -1, -x,
      y,  x, -β
  )

  # Return the Jacobian.
  jac
end

In [ ]:
# Calculate Lyapunov spectra in parallel threads.

ds = CoupledODEs(lorenz_static, u0, p)
tands = TangentDynamicalSystem(ds; J = lorenz_static_jac)

# Preallocate the array for storing the Lyapunov spectra.
λs = zeros(length(params), 3)

# Since `DynamicalSystem`s are mutable, we need to copy to parallelize.
systems = [deepcopy(tands) for _ in 1:Threads.nthreads()-1]
pushfirst!(systems, tands)

Threads.@threads for i in eachindex(params)
    system = systems[Threads.threadid()]
    set_parameters!(system, params[i])
    λs[i, :] .= lyapunovspectrum(system, Int(1e3); Ttr = 1e2)
end

# Display the calculated Lyapunov spectra.
λs

In [ ]:
# Store Leading Lyapunov exponents.
LLEs = [λs[i,1] for i in 1:size(λs)[1]]

# Store Lyapunov dimensions.
LDs = Float64[]
for i in 1:size(λs)[1]
  try
    push!(LDs, lyapunov_dimension(λs[i,:]))
  catch
    push!(LDs, NaN)
  end
end

In [ ]:
# Plot both LLEs and LDs side by side in one figure.
fig = Figure(size = (900, 400))

# LLEs plot.
ax1 = Axis(
  fig[1, 1];
  xlabel = L"\rho",
  ylabel = L"\sigma",
  title = "Leading Lyapunov Exponent"
)
hm1 = heatmap!(
    ax1,
    ρs,
    σs,
    reshape(LLEs, (length(ρs), length(σs))),
    colormap = :thermal
)
Colorbar(fig[1, 2], hm1, label = L"\lambda_1")

# LDs plot.
ax2 = Axis(
  fig[1, 3];
  xlabel = L"\rho",
  ylabel = L"\sigma",
  title = "Lyapunov Dimension"
)
hm2 = heatmap!(
    ax2,
    ρs,
    σs,
    reshape(LDs, (length(ρs), length(σs))),
    colormap = :thermal,
    # Clip LDs below 2.
    colorrange = (max(
      1.95,
      minimum(filter(!isnan, LDs))),
      maximum(filter(!isnan, LDs))
    )
)
Colorbar(fig[1, 4], hm2, label = L"D_L")

fig